In [ ]:
!pip install mpmath

In [ ]:
import pandas
import os

# This query represents dataset "ced family history" for domain "survey" and was generated for All of Us Controlled Tier Dataset v7
dataset_63942080_survey_sql = """
    SELECT
        answer.person_id,
        answer.survey_datetime,
        answer.survey,
        answer.question_concept_id,
        answer.question,
        answer.answer_concept_id,
        answer.answer,
        answer.survey_version_concept_id,
        answer.survey_version_name  
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.ds_survey` answer   
    WHERE
        (
            question_concept_id IN (
                836793
            )
        )  
"""

dataset_63942080_survey_df = pandas.read_gbq(
    dataset_63942080_survey_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_63942080_survey_df.head(5)

In [ ]:
from scipy.stats import chi2_contingency, fisher_exact

import numpy as np

In [ ]:
family_ct = dataset_63942080_survey_df[~dataset_63942080_survey_df['answer'].str.contains('- Self')].value_counts('person_id')

In [ ]:
Ced_cases_survey=dataset_63942080_survey_df[dataset_63942080_survey_df['answer'].str.contains('Self')]['person_id']

In [ ]:
dataset_63942080_survey_df.loc[dataset_63942080_survey_df['person_id'].isin(Ced_cases_survey),'ced']='Celiac'
dataset_63942080_survey_df['ced'].fillna('healthy',inplace=True)

In [ ]:
dataset_63942080_survey_df.drop_duplicates('person_id').value_counts('ced')

In [ ]:
#search for not answered
answered_total=dataset_63942080_survey_df[~dataset_63942080_survey_df['answer'].isin(['PMI: Skip','Including yourself, who in your family has had celiac disease? - Self'])].drop_duplicates('person_id').value_counts('ced')

In [ ]:
#make contingency table
chi_dat=pandas.DataFrame(dataset_63942080_survey_df[dataset_63942080_survey_df['answer']!='PMI: Skip'].value_counts(['ced','answer'])).reset_index().pivot(columns='ced',index='answer',values='count').fillna(0)
chi_dat.loc['No Family','Celiac']=1789-answered_total['Celiac']
chi_dat.loc['No Family','healthy']=103824-answered_total['healthy']
chi_dat.drop('Including yourself, who in your family has had celiac disease? - Self',inplace=True)

In [ ]:
#chi-square by question

obs=chi_dat
res = chi2_contingency(obs)


In [ ]:
import mpmath

In [ ]:
from scipy.stats import chi2
from mpmath import mp

# Set the precision
mp.dps = 800  # Set precision to 500 decimal places

# Degrees of freedom and chi-square value (example values)
df = 6
chi_squared = 3769.6962794742876

# Calculate the p-value using chi2.sf (1 - CDF)
p_value = chi2.sf(chi_squared, df)

# Convert p-value to mpmath format for higher precision display
p_value_mpmath = mp.mpf(p_value)

# Display the p-value with high precision
print(f"P-value for chi-square with df={df} and chi-square={chi_squared}:")
print(p_value_mpmath)
#p=2.42e-802